In [78]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

search = arxiv.Search(
    query = "Attention is all you need",
    max_results = 10,
    sort_by=arxiv.SortCriterion.SubmittedDate, 
    sort_order=arxiv.SortOrder.Descending
)

results = client.results(search)
# `results` is a generator; you can iterate over its elements one by one...
for r in client.results(search):
    print(r.download_pdf())
    print()
    break

./2404.15626v1.An_Electromagnetism_Inspired_Method_for_Estimating_In_Grasp_Torque_from_Visuotactile_Sensors.pdf



In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("assets/arxiv_paper.pdf")
loader.load()

## Инициализация модели
Теперь инициализируем модель GigaChat.

In [46]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

In [47]:
from langchain.chat_models.gigachat import GigaChat

llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("assets/arxiv_paper.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

После нарезки мы получили 91 документ частями книги.

## Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

> Обратите внимание, что сервис для вычисления эмбеддингов может тарифицироваться отдельно от стоимости модели GigaChat.

In [51]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

## Поиск по базе данных

Теперь можно обратиться к базе данных и попросить найти документы, которые с наибольшей вероятностью содержат ответ на наш вопрос.

По-умолчанию база данных возвращает 4 наиболее релевантных документа. Этот параметр можно изменить в зависимости от решаемой задачи и типа документов.

Видно, что первый же документ содержит внутри себя часть книги с ответом на наш вопрос.

In [52]:
docs = db.similarity_search(question, k=4)
len(docs)

4

In [53]:
print(f"... {str(docs[0])[620:800]} ...")

... ение (fork) открытой библиотеки LangСhain на Python. Её главная цель — облегчить жизнь разработчику. Библиотека состоит из большого количества различных компонентов, которые позвол ...


In [ ]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate

propmpt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            #             prompt = """Сгенерируй от {dataset_size_min} до {dataset_size_max} синонимов для слова "{subject}".
            # Результат верни в формате JSON списка без каких либо пояснений, например, ["синоним1", "синоним2", "синоним3", "синоним4"].
            # Не дублируй фразы."""
            prompt=load_prompt("lc://prompts/synonyms/synonyms_generation.yaml")
        )
    ]
)

## QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель и ретривер (база данных).

In [54]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

Наконец можно задать вопрос нашей цепочке и получить правильный ответ!

In [58]:
qa_chain({"query": question})

/workspace/Syrenny/GigaHack/Syrenny/giga_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Какой Loss использует Yolov8?',
 'result': 'Я не знаю ответа на этот вопрос.'}

Несколько дополнительных вопросов для проверки работоспособности:

In [89]:
qa_chain({"query": "Расскажи про тимлида Сбера"})

{'query': 'Расскажи про тимлида Сбера',
 'result': 'Извините, но у меня нет информации о конкретном тимлиде Сбера. Могу рассказать вам о том, что такое тимлид и какие у него обязанности. Тимлид - это лидер команды, который отвечает за управление командой, ее развитие и достижение поставленных целей. Он координирует работу членов команды, помогает им решать возникающие проблемы, поддерживает мотивацию и вовлеченность. Кроме того, тимлид также участвует в принятии решений, связанных с проектами, и помогает команде в достижении общих целей.'}

In [24]:
qa_chain({"query": "Основные особенности Yolov8"})

{'query': 'Основные особенности Yolov8',
 'result': 'YOLOv8 имеет несколько особенностей, которые отличают его от других версий YOLO. Во-первых, он использует новую архитектуру, которая сочетает в себе модули FAN и PAN. Это позволяет ему лучше захватывать особенности на разных масштабах и разрешениях, что важно для точного обнаружения объектов разного размера и формы. Кроме того, YOLOv8 превосходит YOLOv5 по нескольким параметрам, включая более высокую метрику mAP и меньшее количество выбросов при измерении против RF100. Он также превосходит YOLOv5 для каждого RF100 категории.'}

In [125]:
import arxiv
import os
import json

def download_from_arxiv(key_word, max_results=10, destination_path="assets/arxiv/", saved_list_path="assets/arxiv/papers.json"):
    client = arxiv.Client()

    search = arxiv.Search(
        query = str(key_word),
        max_results = max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate, 
        sort_order=arxiv.SortOrder.Descending
    )
    # Проверка существования файла с сохраненным списком скачанных arXiv ID
    if os.path.exists(saved_list_path):
        with open(saved_list_path, "r") as f:
            downloaded_arxiv_ids = json.load(f)
    else:
        downloaded_arxiv_ids = []
    
    for result in client.results(search):
        # Проверка наличия статьи в списке уже скачанных
        if result.entry_id.split("/")[-1] in downloaded_arxiv_ids:
            print(f"Статья {result.entry_id} уже скачана и пропущена.")
            continue
        
        # Скачивание PDF-файла статьи
        pdf_path = result.download_pdf(dirpath=destination_path)
        if pdf_path:
            print(f"Статья {result.entry_id} успешно скачана и сохранена в {pdf_path}")
            # Добавление arXiv ID в список скачанных
            downloaded_arxiv_ids.append(result.entry_id.split("/")[-1])
    # Сохранение списка скачанных arXiv ID в файл JSON
    with open(saved_list_path, "w") as f:
        json.dump(downloaded_arxiv_ids, f)

In [127]:
key_words = ["NLP", "RAG", "ChatBot", "LLM", "Speech Recognition", "LangChain", "LLM Agents"]

for key_word in key_words:
    # Пример использования функции
    download_from_arxiv(key_word, max_results=10)

Статья http://arxiv.org/abs/2404.15238v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.15104v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14977v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14963v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14943v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.15382v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14809v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14740v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14695v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14631v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.15488v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14043v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.13948v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.13892v2 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.13781v1 уже скачана и пропущена.
Статья htt

In [118]:
def get_pdf_files(directory):
    pdf_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

In [119]:
# Пример использования функции
directory = "assets/arxiv/"
pdf_files = get_pdf_files(directory)
for pdf_file in pdf_files:
    print(pdf_file)

assets/arxiv/2404.15604v1.Hybrid_LLM_Rule_based_Approaches_to_Business_Insights_Generation_from_Structured_Data.pdf
assets/arxiv/2404.15588v1.Minimal_Evidence_Group_Identification_for_Claim_Verification.pdf
assets/arxiv/2404.15578v1.Can_Foundational_Large_Language_Models_Assist_with_Conducting_Pharmaceuticals_Manufacturing_Investigations_.pdf
assets/arxiv/2404.15564v1.Guided_AbsoluteGrad__Magnitude_of_Gradients_Matters_to_Explanation_s_Localization_and_Saliency.pdf
assets/arxiv/2404.15592v1.ImplicitAVE__An_Open_Source_Dataset_and_Multimodal_LLMs_Benchmark_for_Implicit_Attribute_Value_Extraction.pdf
assets/arxiv/2404.13892v2.Retrieval_Augmented_Audio_Deepfake_Detection.pdf
assets/arxiv/2404.14631v1.Learning_Word_Embedding_with_Better_Distance_Weighting_and_Window_Size_Scheduling.pdf
assets/arxiv/2404.15552v1.Cross_Temporal_Spectrogram_Autoencoder__CTSAE___Unsupervised_Dimensionality_Reduction_for_Clustering_Gravitational_Wave_Glitches.pdf
assets/arxiv/2404.14901v1.Beyond_Code_Generation

In [120]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader


def summarize(path_to_pdf_file, chunk_size=5000, chunk_overlap=1000): 
    loader = PyMuPDFLoader(path_to_pdf_file)
    docs = loader.load()
    split_docs = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap).split_documents(
        docs
    )
    
    giga = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)
    chain = load_summarize_chain(giga, chain_type="map_reduce")
    res = chain.run(split_docs)
    return res

In [131]:
import sqlite3

class ArticlesDB:
    def __init__(self, db_path='articles.db'):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS articles
                            (id INTEGER PRIMARY KEY,
                            title TEXT NOT NULL,
                            summary TEXT)''')
        self.conn.commit()

    def add_article(self, title, summary):
        self.cursor.execute('INSERT INTO articles (title, summary) VALUES (?, ?)', (title, summary))
        self.conn.commit()

    def delete_article(self, title):
        self.cursor.execute('DELETE FROM articles WHERE title = ?', (title,))
        self.conn.commit()

    def get_article_summary(self, title):
        self.cursor.execute('SELECT summary FROM articles WHERE title = ?', (title,))
        result = self.cursor.fetchone()
        if result:
            return result[0]
        else:
            return None

    def get_all_articles(self):
        self.cursor.execute('SELECT title, summary FROM articles')
        return self.cursor.fetchall()

    def close(self):
        self.conn.close()


In [136]:
articles_db = ArticlesDB(db_path="articles.db")

In [ ]:
from tqdm import tqdm

for filename in tqdm(pdf_files, desc="Processing PDF files"):
    article_title = filename.split("/")[-1][:-4]
    article_summary = articles_db.get_article_summary(article_title)
    if article_summary is None:
        result = summarize(filename,
                       chunk_size=5000,
                       chunk_overlap=1000)
        articles_db.add_article(article_title, result)
        print(f"Article \"{article_title}\" saved to database")
print(articles_db.get_all_articles())

In [143]:
for name, summary in articles_db.get_all_articles():
    print(name)
    print(summary)
    print()

2404.15604v1.Hybrid_LLM_Rule_based_Approaches_to_Business_Insights_Generation_from_Structured_Data
The paper discusses the integration of rule-based systems and Large Language Models (LLMs) to generate actionable business insights from structured data. The hybrid approach combines the robustness of rule-based systems with the adaptive power of LLMs, addressing the complexities of data extraction in the field of business intelligence. The hybrid approach combines interpretable AI techniques, such as LIME, rule-based systems, and supervised document classification, to extract actionable insights from business data. LLM plays a vital role in this process by enhancing the understanding and generation of natural language-generated insights. The hybrid approach can uncover personalized and nuanced user interests, needs, and goals from user journeys and user activities on the platform. However, when implementing this approach, it is essential to consider data quality, domain knowledge, and sc

In [ ]:
from langchain.chat_models.gigachat import GigaChat

with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()
    
llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = PyPDFDirectoryLoader("assets/arxiv/")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

In [ ]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())